In [ ]:
import pandas as pd
data = pd.read_excel('./attach_02.xlsx')
# 데이터 파일 로드
print(data[:3])
# 데이터 형식 확인

In [ ]:
del data['No']
# No 열 삭제
labels = data['신고유형'].unique()
# 신고유형 원문 저장
data['신고유형'] = data['신고유형'].replace(labels, list(range(0, len(labels))))
# 신고유형을 정수로 변환
print(data[:3])

In [ ]:
from konlpy.tag import Kkma
import re
# 형태소 분석기, 정규식 임포트
kkma = Kkma()
matcher = re.compile(r'^[가-힣]+$')
# 형태소 분석기 생성, 한글로만 이루어진 문자열 추출하는 정규식 컴파일

In [ ]:
y_data = []
for number in data['신고유형']:
    y_data.append([number])

X_data = []
# 한글로만 이루어진 문자열 추출
for sentence in data['내용']:
    temp = []
    nouns = kkma.nouns(sentence)
    for noun in nouns:
        if matcher.match(noun):
            temp.append(noun)
    X_data.append(temp)
# 토큰화하여 저장

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_data)
# 토큰을 정수화

In [ ]:
threshold = 2
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)
print('제한할 단어 집합의 크기: %s'%(total_cnt - rare_cnt + 1))

# 1번만 등장한 단어를 분석에서 제외함
# https://wikidocs.net/44249

In [ ]:
tokenizer  = Tokenizer(total_cnt - rare_cnt + 1)
tokenizer.fit_on_texts(X_data)
X_data = tokenizer.texts_to_sequences(X_data)
# 내용문장을 정수화

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_data = pad_sequences(X_data, maxlen = 150)
# 내용의 길이를 150으로 맞춤

In [ ]:
trainLen = round(len(data) * 0.8)
X_train = X_data[:trainLen]
X_test = X_data[trainLen:]
y_train = y_data[:trainLen]
y_test = y_data[trainLen:]
# 8:2 비율로 학습데이터와 검증데이터를 분리

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train)
# 나이브 베이즈 학습

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.utils import to_categorical

y_train_ot = to_categorical(y_train)
y_test_ot = to_categorical(y_test)
# 레이블 데이터 원-핫 인코딩

rnn = Sequential()
rnn.add(Embedding(total_cnt - rare_cnt + 1,  32, mask_zero=True))
rnn.add(LSTM(32))
rnn.add(Dense(len(labels), activation='softmax'))
rnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
rnn.summary()
# rnn 생성

In [ ]:
rnn.fit(X_train, y_train_ot, validation_data=(X_test, y_test_ot), epochs=200, batch_size=128, verbose=1)
# LSTM 학습

In [ ]:
import numpy as np

def encodeSentence(sentence):
    temp = []
    nouns = kkma.nouns(sentence)
    for noun in nouns:
        if matcher.match(noun):
            temp.append(noun)

    temp = tokenizer.texts_to_sequences([temp])
    temp = pad_sequences(temp, maxlen = 150)

    return temp

def predictByNB(encodedSentence):
    return labels[nb.predict(encodedSentence)[0]]

def predictByRNN(encodedSentence):
    pred = rnn.predict(encodedSentence)
    labelIndex = np.argmax(pred[0])
    return labels[labelIndex]

def predict(sentence):
    encodedSentence = encodeSentence(sentence)
    nb = predictByNB(encodedSentence)
    rnn = predictByRNN(encodedSentence)
    print('나이브베지안:\t%s\nRNN 예측:\t%s'%(nb, rnn))

In [ ]:
testStr = '직업훈련학원 대표 F는 지방자치단체로부터 위탁받은 지역실업자훈련과정을 운영하면서 훈련생이 아닌 자들이 마치 훈련받은 것처럼 서류를 허위로 꾸며매월 훈련비를 편취하였고, 학원에 훈련생들의 명의만을 허위로 등록한 후 지방자치단체로부터 매월 지급받는 훈련수당과 근로자 수강지원비를 편취함.'

predict(testStr)

In [ ]:
testStr = '모 공직유관단체 직원 A 등은 지역본부장 B의 지시로 지역본부장의 가족이 운영하는 가게에서 선물을 구입하고, 사업의 원활한 추진을위하여 업무관계자에게 이를 제공하는 과정에서 본부장의 부당한 지시를 거부하거나 행동강령책임관과 상담하지 않고 그대로 따름'

predict(testStr)

In [ ]:
testStr = '모 중앙행정기관 사무관 A는 소속 기관의 무기계약직 채용심사 과정에서 자신의 조카가 응시한 사실을 알면서도 심사절차를 회피하지 않고 면접 심사위원으로 참여'

predict(testStr)

In [ ]:
testStr = '중앙부처 소속 국장 B의 자녀 A가 ○○지방자치단체에서 실시하는 변호사 자격소지자 제한경쟁 채용시험에 응시하였음. 국장 B는 자녀 A 몰래 면접위원인 인사과장 C에게 면접시험 점수를 높게 주어 합격시켜 달라는 청탁을 하였고 면접위원으로 참석한 인사과장 C가 면접시험 점수를 높게 주어 자녀 A가 합격한 경우'

predict(testStr)